Install dependencies


In [ ]:
!pip install torch torchvision
!pip install Pillow
!pip install imgaug
!pip install tensorboard

##  Load TensorBoard Extension

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

download the dataset from drive


In [ ]:
!unzip '/content/drive/MyDrive/data_lego.zip' -d '/content/data'

In [ ]:
!nvidia-smi

code


In [ ]:
import os
import numpy as np
from PIL import Image
import pandas as pd
import cv2

import argparse
import glob
import datetime
from tqdm import tqdm


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F

# from torch.utils.tensorboard import SummaryWriter

import imgaug.augmenters as iaa
import imgaug as ia

import matplotlib.pyplot as plt

from torch.utils.tensorboard import SummaryWriter

# Configuration
class Config:
    OUTPUT = "/content/drive/MyDrive"
    DATASET_PATH = "/content/data"
    LABELS_PATH = os.path.join(DATASET_PATH, "data.csv")
    IMAGES_PATH = os.path.join(DATASET_PATH, "images")
    SUMMARY_PATH = os.path.join(DATASET_PATH, "summary")
    DEBUG_PATH = os.path.join(DATASET_PATH, "examples")
    IMAGE_RESIZE = 256
    AUGMENTATION_FACTOR = 30
    TRAIN_SPLIT= 0.8
    BATCH_SIZE = 32
    FREEZE_BACKBONE = True
    EPOCHS = 60
    DEBUG = True
    MODEL_NAME = "ckpt_{epoch:d}_{loss:f}.pth"
    CKPT_SAVE_INTERVAL = 5
    LR = 1e-4
    EPS = 1e-6



## Lego dataset definition

In [ ]:
class LegoDataset(Dataset):

    def __init__(self, csv_file, root_dir, rgb_transform=None,mask_transform=None, do_aug=False, augmentation_factor=1):
        self.data = pd.read_csv(os.path.join(root_dir,csv_file))
        self.root_dir = root_dir
        self.rgb_transform = rgb_transform
        self.mask_transform = mask_transform
        self.do_aug = do_aug
        self.brick_type_dict = {
                        '6*2': 0,
                        '3*1': 1,
                        '3*2': 2,
                        '2*1': 3,
                        '4*2': 4,
                        '1*1': 5,
                        '8*2': 6,
                        '4*1': 7,
                        '2*1_pyramid': 8,
                        '2*2': 9,
                        '6*1': 10
                    }
        
        self.seq_train = iaa.Sequential([
            iaa.CropAndPad(percent=(0, 0.1)),
            iaa.Affine(rotate=(-20, 20)),
            iaa.Resize({"height": Config.IMAGE_RESIZE, "width": Config.IMAGE_RESIZE}),
            iaa.Affine(shear=(-15, 15)),
            iaa.Affine(translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}),
            iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}),
            iaa.Flipud(0.3),
            iaa.Fliplr(0.3)
        ])

        self.seq_color = iaa.Sequential([
            iaa.MultiplyAndAddToBrightness(mul=(0.5, 1.5), add=(-30, 30)),
            iaa.GammaContrast((0.5, 2.0), per_channel=True),
            iaa.GaussianBlur((0, 3.0)),
            iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
            iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
            iaa.LinearContrast((0.75, 1.5)),
            iaa.Multiply((0.8, 1.2), per_channel=0.2),
            iaa.AddToHueAndSaturation((-20, 20))
        ])
        
    

        self.seq_test = iaa.Sequential([
            iaa.Resize({"height": Config.IMAGE_RESIZE, "width": Config.IMAGE_RESIZE})
        ])

        self.augmentation_factor = augmentation_factor
        self.do_aug = do_aug

    def __len__(self):
        return len(self.data) * self.augmentation_factor

    def __getitem__(self, idx):

        # Determine the original index in the annotations
        original_index = idx // self.augmentation_factor
        sample = self.data.iloc[original_index]

        brick_name = sample['filename']
        img_path = os.path.join(self.root_dir,'images', f"{brick_name}_rgb.png")
        image = np.array(Image.open(img_path))[..., :3]

        # Load the binary mask for each image
        mask_path = os.path.join(self.root_dir, 'images', f"{brick_name}_mask.png")
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
        mask = mask[..., np.newaxis]  # Add an extra channel dimension at the end
        mask = mask / 255.  # Normalize to [0, 1]

        

        # Extract target variables
        brick_type = self.brick_type_dict[sample['brick_type']]

       
        # Apply augmentations
        if self.do_aug:
            # Augment both image and mask
            augmentations = self.seq_train.to_deterministic()  # Ensures both image and mask undergo the same augmentations
            image = augmentations.augment_image(image)
            mask = (augmentations.augment_image(mask) > 0.5).astype(np.uint8) #  Ensures that after the spatial augmentations, mask remains strictly binary

            # Apply color-related augmentations only to the image
            image = self.seq_color.augment_image(image)
        else :
            image = self.seq_test(image=image)
            mask = self.seq_test(image=mask)
        
        image = Image.fromarray(image)
        mask = Image.fromarray((mask.squeeze() * 255).astype(np.uint8))

        if self.rgb_transform:
            image = self.rgb_transform(image)
        
        if self.mask_transform:
            mask = self.mask_transform(mask)
        
        return (
            image,
            mask,
            torch.tensor(brick_type).long()
        )

## Early stop system

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, delta=0, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta

    def __call__(self, val_loss):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

## UNet architecture Model

In [ ]:
class LegoModelUNet(nn.Module):
    def __init__(self, in_channels, num_classes = 11, inference = True):
        super(LegoModelUNet, self).__init__()
        
        self.inference = inference

        # Contracting Path
        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)
        
        # Bottleneck
        self.bottleneck = self.conv_block(512, 1024)
        
        # Expansive Path
        self.upconv4 = self.upconv_block(1024, 512)
        self.upconv3 = self.upconv_block(512, 256)
        self.upconv2 = self.upconv_block(256, 128)
        self.upconv1 = self.upconv_block(128, 64)
        
        # Segmentation head
        self.segmentation_head = nn.Sequential(
            nn.Conv2d(64, 1, kernel_size=1),
            nn.Sigmoid()
        )

        
        # Classification head
        self.global_avg_pooling = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, num_classes)
        
    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
    
    def upconv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Contracting Path
        e1 = self.enc1(x)
        e2 = self.enc2(F.max_pool2d(e1, 2))
        e3 = self.enc3(F.max_pool2d(e2, 2))
        e4 = self.enc4(F.max_pool2d(e3, 2))
        
        # Bottleneck
        b = self.bottleneck(F.max_pool2d(e4, 2))
        
        # Expansive Path
        d4 = self.upconv4(b)
        d3 = self.upconv3(d4 + e4)
        d2 = self.upconv2(d3 + e3)
        d1 = self.upconv1(d2 + e2)
        
        # Segmentation head
        seg_output = self.segmentation_head(d1 + e1)
        
        # Classification head
        avg_pool = self.global_avg_pooling(b).view(b.size(0), -1)
        fc1 = F.relu(self.fc1(avg_pool))
        class_output = F.softmax(self.fc2(fc1), dim=1)
        
        return seg_output, class_output


In [ ]:
def visualizeDataset(csv_file,root_dir,rgb_transform,mask_transform,do_aug,augmentation_factor) :
    # Load the dataset
    dataset = LegoDataset('data.csv', './data', rgb_transform=rgb_transform, mask_transform = mask_transform, do_aug=do_aug, augmentation_factor=augmentation_factor)

    # Define the inverse normalization function
    inv_normalize = transforms.Normalize(
                        mean = [ -0.485/0.229, -0.456/0.224, -0.406/0.225 ],
                        std = [ 1/0.229, 1/0.224, 1/0.225 ])
    
      
    # Randomly select some samples
    indices = np.random.choice(len(dataset), 5)

    for index in indices:

        image, mask, _ = dataset[index]

        # Convert tensors to numpy arrays for visualization
        if isinstance(image, torch.Tensor):
            image = inv_normalize(image)  # Apply the inverse normalization
            image = image.permute(1, 2, 0).numpy()
        if isinstance(mask, torch.Tensor):
            mask = mask.squeeze().numpy()

        # Check the min and max of the image
        print(f"Image min: {image.min()}, max: {image.max()}")

        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        axes[0].imshow(image)
        axes[0].set_title("RGB Image")
        axes[1].imshow(mask, cmap='gray')
        axes[1].set_title("Mask")
        plt.show()

## Training and Evaluation Functions

In [ ]:
def train(model, data_loader, optimizer, device, epoch):

    data_loader.dataset.dataset.do_aug = True
    data_loader.dataset.dataset.augmentation_factor = Config.AUGMENTATION_FACTOR

    model.train()

    train_loss = 0.0
    train_iou = 0.0
    
    pbar = tqdm(data_loader)

    for batch_idx, (
        image,
        mask,
        brick_type
    ) in enumerate(pbar, 0):
        
        image = image.to(device)
        mask = mask.to(device)
        brick_type = brick_type.to(device)
        
        # Optimizer zero
        optimizer.zero_grad()
        
        # Model inference
        out_mask, out_brick_type = model(image)
        
        
        if len(mask.shape) == 5:
            mask = mask.squeeze(2).squeeze(3)

        # Calculate loss
        loss_brick = criterion_brick_type(out_brick_type, brick_type)
        loss_mask = F.binary_cross_entropy(out_mask, mask)   # BCE loss for segmentation
        
        # Combine the losses
        loss = loss_brick + loss_mask

        # Compute IoU
        iou = compute_iou(out_mask, mask)

        # Backward pass and optimization
        loss.backward()
        
        # Optimizer
        optimizer.step()

        train_loss += loss.item()
        train_iou += iou
        
    avg_loss = train_loss / len(data_loader)
    avg_iou = train_iou / len(data_loader)

    return avg_loss, avg_iou

def test(model, data_loader, device, epoch):

    data_loader.dataset.dataset.do_aug = False
    data_loader.dataset.dataset.augmentation_factor = 1

    model.eval()

    test_loss_brick_type = 0.0
    test_loss = 0.0
    test_iou = 0.0
    
    # Initialize accumulators
    total_type_correct = 0
   

    pbar = tqdm(data_loader)

    with torch.no_grad():

        for batch_idx, (
            image,
            mask,
            brick_type
        ) in enumerate(pbar, 0):
            
            image = image.to(device)
            mask = mask.to(device)
            brick_type = brick_type.to(device)
       
            # Forward pass
            out_mask, out_brick_type = model(image)

           # Calculate loss
            loss_brick = criterion_brick_type(out_brick_type, brick_type)
            loss_mask = F.binary_cross_entropy(out_mask, mask)   # BCE loss for segmentation
            
            # Combine the losses
            loss = loss_brick + loss_mask

            # Compute IoU
            iou = compute_iou(out_mask, mask)

            pbar.set_description(f"TEST loss={float(loss)}")

            
            test_loss += loss.item()
            test_iou += iou
            
            # Brick accuracy
            type_correct = (torch.argmax(out_brick_type, dim=1) == brick_type).float().sum()
            total_type_correct += type_correct

     
    avg_loss = test_loss / len(data_loader)
    avg_iou = test_iou / len(data_loader)

    # return avg_loss, loss_brick, loss_rotation, loss_color, type_correct
    return avg_loss, avg_iou, type_correct

def criterion_values(pred, true):
    return F.mse_loss(torch.sigmoid(pred), true)
  
criterion_brick_type = nn.CrossEntropyLoss()


def compute_iou(pred_mask, true_mask):
    # True Positive: Predicted = 1, Ground Truth = 1
    intersection = (pred_mask * true_mask).float().sum()
    
    # True Positive + False Positive + False Negative
    union = pred_mask.float().sum() + true_mask.float().sum() - intersection

    iou = intersection / union
    return iou.item()

# Define the data transformations
rgb_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

mask_transform = transforms.Compose([
    transforms.ToTensor(),
])

In [ ]:

def main(experiment):
    
    torch.manual_seed(108)
    np.random.seed(108)

    writer = SummaryWriter(f"runs/{experiment}")

    # define Device
    if torch.cuda.is_available():
      device = torch.device("cuda")
      print("CUDA is available. Using GPU.")
    else:
      device = torch.device("cpu")
      print("CUDA is not available. Using CPU.")

    # Create project folder and names
    OUTPUT_FOLDER = os.path.join(Config.OUTPUT, experiment, datetime.datetime.now().strftime("%d_%m_%Y__%H_%M_%S"))
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    print('OUTPUT_FOLDER',OUTPUT_FOLDER)



    os.makedirs(Config.SUMMARY_PATH, exist_ok=True)
    os.makedirs(Config.DEBUG_PATH, exist_ok=True)

    CHECKPOINTS_FOLDER = os.path.join(OUTPUT_FOLDER, 'checkpoints')
    os.makedirs(CHECKPOINTS_FOLDER, exist_ok=True)

    # Define the data transformations
    data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Define the dataset and data loader
    lego_dataset = LegoDataset('data.csv', './data', rgb_transform=rgb_transform, mask_transform = mask_transform, do_aug=True, augmentation_factor=Config.AUGMENTATION_FACTOR)
    train_size = int(Config.TRAIN_SPLIT * len(lego_dataset))
    test_size = len(lego_dataset) - train_size
    print('train_size',train_size)
    print('test_size',test_size)
    train_dataset, test_dataset = torch.utils.data.random_split(lego_dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=Config.BATCH_SIZE, shuffle=False)

    # Initialize the model and define the loss function and optimizer
    model = LegoModelUNet(in_channels=3, inference=False)

    optimizer = optim.Adam(model.parameters(), lr=Config.LR, eps = Config.EPS)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    model.to(device)

    # Epochs
    best_val_loss = float('inf')
    best_epoch = 0
    best_model = None
    best_type_accuracy = None

    # Initialize accumulators
    train_losses = []
    val_losses = []
    epochs_type_accuracy  = []


    early_stopping = EarlyStopping(patience=10, verbose=True)

    # Train the model
    for epoch in range(Config.EPOCHS):
        
        
        train_loss, train_iou = train(model, train_loader, optimizer, device, epoch)
        val_loss, val_iou, total_type_correct = test(model, test_loader, device, epoch)

        writer.add_scalar('Training Loss', train_loss, epoch)
        writer.add_scalar('Validation Loss', val_loss, epoch)
        writer.add_scalar('Training IoU', train_iou, epoch)
        writer.add_scalar('Validation IoU', val_iou, epoch)
        writer.add_scalar('Validation Accuracy', total_type_correct, epoch)


        
        # Compute average metrics for the epoch
        epoch_type_accuracy = total_type_correct / len(lego_dataset) * 100

        # Save the best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_epoch = epoch
            best_model = model.state_dict()
            best_gender_accuracy = epoch_type_accuracy
        
        # Checkpointing
        if epoch > 0 and epoch % Config.CKPT_SAVE_INTERVAL == 0 :
            print(f"Saved checkpoint {epoch}\n")
            model_path_solve = os.path.join(CHECKPOINTS_FOLDER, f"ckpt_{epoch}_{val_loss}.pth")
            optimizer_path_solve = os.path.join(CHECKPOINTS_FOLDER, model_path_solve.replace('ckpt_', 'optim_'))
            torch.save(model.state_dict(), model_path_solve)
            torch.save(optimizer.state_dict(), optimizer_path_solve)

        # Print progress
        print(f"Epoch {epoch + 1}/{Config.EPOCHS}\n"
          f"Type accuracy: {epoch_type_accuracy:.2f}%\n"
          f"Validation loss: {val_loss:.4f}")
        

        # Step the scheduler
        scheduler.step()

        # Check for early stopping
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping")
            break
        
       

    # Save the best model
    torch.save(best_model, os.path.join(CHECKPOINTS_FOLDER, f"best_{best_epoch}_{best_val_loss}.pth"))
        

In [ ]:
main('model_lego_sorter_unet')